In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time
import matplotlib.pyplot as plt


In [ ]:
examples, metadata = tfds.load('wmt14_translate/fr-en', with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

# Define the tokenizer for English using the subword tokenizers from TFDS



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

...

Generating validation examples...:   0%|          | 0/3000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/wmt14_translate/fr-en/1.0.0.incomplete8NN2U1/wmt14_translate-validation.tf…

Generating test examples...:   0%|          | 0/3003 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/wmt14_translate/fr-en/1.0.0.incomplete8NN2U1/wmt14_translate-test.tfrecord…

Dataset wmt14_translate downloaded and prepared to /root/tensorflow_datasets/wmt14_translate/fr-en/1.0.0. Subsequent calls will reuse this data.


In [ ]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for en, fr in train_examples), target_vocab_size=2**13)

# Define the tokenizer for French using the subword tokenizers from TFDS
tokenizer_fr = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (fr.numpy() for en, fr in train_examples), target_vocab_size=2**13)

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

def encode(lang1, lang2):
    lang1 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
        lang1.numpy()) + [tokenizer_en.vocab_size+1]

    lang2 = [tokenizer_fr.vocab_size] + tokenizer_fr.encode(
        lang2.numpy()) + [tokenizer_fr.vocab_size+1]

    return lang1, lang2

def tf_encode(en, de):
    result_en, result_de = tf.py_function(encode, [en, de], [tf.int64, tf.int64])
    result_en.set_shape([None])
    result_de.set_shape([None])

    return result_en, result_de

train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.padded_batch(BATCH_SIZE)

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1


In [ ]:
"""class InstanceNormalization(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-6):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale',
                                     shape=input_shape[-1:],
                                     initializer='ones',
                                     trainable=True)
        self.offset = self.add_weight(name='offset',
                                      shape=input_shape[-1:],
                                      initializer='zeros',
                                      trainable=True)

    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (x - mean) * inv
        return self.scale * normalized + self.offset

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, model_dim, num_heads, dff, normalization_layer, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.mha = MultiHeadAttention(num_heads, model_dim)
        self.ffn = self.point_wise_feed_forward_network(model_dim, dff)

        # Use the provided normalization layer
        self.norm1 = normalization_layer
        self.norm2 = normalization_layer

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        attn_output = self.mha(x, x, x)  # q, k, v are all the same for self-attention
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(x + attn_output)  # Apply normalization

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.norm2(out1 + ffn_output)  # Apply normalization

        return out2

    def point_wise_feed_forward_network(self, model_dim, dff):
        return tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
            tf.keras.layers.Dense(model_dim)  # (batch_size, seq_len, model_dim)
        ])"""

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, model_dim):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.model_dim = model_dim

        assert model_dim % num_heads == 0  # Ensure the model dimension is divisible by the number of heads

        self.depth = model_dim // num_heads

        self.Wq = self.add_weight(shape=(model_dim, model_dim), initializer='random_normal')
        self.Wk = self.add_weight(shape=(model_dim, model_dim), initializer='random_normal')
        self.Wv = self.add_weight(shape=(model_dim, model_dim), initializer='random_normal')

        self.dense = tf.keras.layers.Dense(model_dim)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)"""
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]

        q = tf.matmul(q, self.Wq)
        k = tf.matmul(k, self.Wk)
        v = tf.matmul(v, self.Wv)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        matmul_qk = tf.matmul(q, k, transpose_b=True)
        depth = tf.cast(self.depth, tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)

        output = tf.matmul(attention_weights, v)
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reshape(output, (batch_size, -1, self.model_dim))

        output = self.dense(output)

        return output





class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, model_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=model_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=model_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

def get_angles(pos, i, model_dim):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(model_dim))
    return pos * angle_rates

def positional_encoding(position, model_dim):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(model_dim)[np.newaxis, :],
                            model_dim)

    # Apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # Apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, model_dim, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(num_heads, model_dim)
        self.ffn = self.point_wise_feed_forward_network(model_dim, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        attn_output = self.mha(x, x, x)  # Self attention (V, K, Q are the same)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # Add & norm

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # Add & norm

        return out2

    def point_wise_feed_forward_network(self, model_dim, dff):
        return tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(model_dim)
        ])

class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, model_dim, num_heads, dff, input_vocab_size,
                 maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()
        self.model_dim = model_dim
        self.num_layers = num_layers

        self.embedding = TokenAndPositionEmbedding(maximum_position_encoding, input_vocab_size, model_dim)
        self.pos_encoding = positional_encoding(maximum_position_encoding, model_dim)

        self.enc_layers = [EncoderLayer(model_dim, num_heads, dff, rate)
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        seq_len = tf.shape(x)[1]

        # Adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, model_dim)
        x *= tf.math.sqrt(tf.cast(self.model_dim, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training)

        return x  # (batch_size, input_seq_len, model_dim)

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, model_dim, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()
        self.mha1 = MultiHeadAttention(num_heads, model_dim)  # Masked self-attention
        self.mha2 = MultiHeadAttention(num_heads, model_dim)  # Encoder-decoder attention

        self.ffn = self.point_wise_feed_forward_network(model_dim, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        # Masked multi-head attention (self-attention)
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        # Multi-head attention (encoder-decoder attention)
        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        # Point-wise feed-forward networks
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

    def point_wise_feed_forward_network(self, model_dim, dff):
        return tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(model_dim)
        ])

class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, model_dim, num_heads, dff, target_vocab_size,
                 maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()
        self.model_dim = model_dim
        self.num_layers = num_layers

        self.embedding = TokenAndPositionEmbedding(maximum_position_encoding, target_vocab_size, model_dim)
        self.pos_encoding = positional_encoding(maximum_position_encoding, model_dim)

        self.dec_layers = [DecoderLayer(model_dim, num_heads, dff, rate)
                           for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, model_dim)
        x *= tf.math.sqrt(tf.cast(self.model_dim, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                   look_ahead_mask, padding_mask)

            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        # x.shape == (batch_size, target_seq_len, model_dim)
        return x, attention_weights

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, model_dim, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(num_layers, model_dim, num_heads, dff,
                               input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, model_dim, num_heads, dff,
                               target_vocab_size, pe_target, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask,
             look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training)  # (batch_size, inp_seq_len, model_dim)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

        return final_output, attention_weights

#layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
#batch_norm = tf.keras.layers.BatchNormalization()

#transformer_block_ln = TransformerBlock(model_dim=512, num_heads=8, dff=2048, normalization_layer=layer_norm)
#transformer_block_bn = TransformerBlock(model_dim=512, num_heads=8, dff=2048, normalization_layer=batch_norm)



In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)  # Ensure d_model is float32
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)  # Ensure warmup_steps is float32

    def __call__(self, step):
        step = tf.cast(step, tf.float32)  # Ensure step is float32
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [ ]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
    # Target is divided into tar_inp (input) and tar_real (target)
    # tar_inp is passed as input to the decoder. tar_real is that same input shifted by 1:
    # At each location in tar_input, tar_real contains the next token that should be predicted.
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp, tar_inp,
                                     True,
                                     enc_padding_mask,
                                     combined_mask,
                                     dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)


In [ ]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          tokenizer_en.vocab_size + 2, tokenizer_fr.vocab_size + 2,
                          pe_input=1000, pe_target=1000, rate=dropout_rate)

checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# If a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')

EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    # Training loop - using batches of 64
    for (batch, (inp, tar)) in enumerate(train_dataset):
        train_step(inp, tar)

        if batch % 50 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')


NameError: name 'tokenizer_en' is not defined